In [34]:
import numpy as np
import pandas as pd
import os
import pickle
import time
import lightgbm as lgb
import matplotlib.pylab as plt

from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV
# from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score


In [35]:
# 1load preprocessed data
train_path = './input/oppo_round1_train_20180929.txt'
valid_file = './input/oppo_round1_vali_20180929.txt'
test_A_file = './input/oppo_round1_test_A_20180929.txt'

train_preprocessed_file = './preprocessed/train_preprocessed.pkl'
valid_preprocessed_file = './preprocessed/valid_preprocessed.pkl'
test_A_preprocessed_file = './preprocessed/test_A_preprocessed.pkl'

# train_df, valid_df, test_df = load_data(train_file, train_preprocessed_file, valid_file, valid_preprocessed_file, test_A_file, test_A_preprocessed_file)

In [36]:
# 1. data preprocessing and save
train_df = pd.read_csv(train_path, sep='\t', header=None,
                               names=['prefix', 'query_prediction', 'title', 'tag', 'label'])

c:\program files\python37\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
train_df.sample(10)

,prefix,query_prediction,title,tag,label
1155037,炝,"{""炝锅鱼的做法"": ""0.047"", ""炝锅面的做法"": ""0.023"", ""炝腰花"": ...",炝土豆丝的做法,菜谱,1
1293529,陆游,"{""陆游简介"": ""0.031"", ""陆游钗头凤"": ""0.044"", ""陆游的诗"": ""0...",陆游诗词,阅读,0
1816395,爱奇艺,"{""爱奇艺视频"": ""0.025"", ""爱奇艺官网"": ""0.047"", ""爱奇艺下截"": ...",爱奇艺,网站,0
103198,狂武战,"{""狂武战帝txt"": ""0.025"", ""狂武战尊"": ""0.064"", ""狂武战帝"": ...",狂武战帝,百科,0
519807,胃疼,"{""胃疼怎么缓解"": ""0.017"", ""胃疼吃什么药"": ""0.059"", ""胃疼怎么办简...",胃疼宁,百科,0
555125,一米,"{""一米滴答物流单号查询"": ""0.067"", ""一米六男生好恶心"": ""0.023"", ""...",九岁孩子一米一,健康,0
885047,积食,"{""积食咳嗽"": ""0.013"", ""积食发烧"": ""0.107"", ""积食怎么办"": ""0...",积食,百科,0
1070893,水印,"{""水印相机安卓版下载"": ""0.025"", ""水印相机怎么改时间"": ""0.030"", ""...",水印相机,应用,1
914588,尿酮体+1,{},尿蛋白质+1尿酮体+1尿胆素原+1,健康,0
1451983,手机铃,"{""手机铃声大全"": ""0.178"", ""手机铃声下载"": ""0.080"", ""手机铃声"":...",手机铃声,歌手,1


In [38]:
train_df.head()

,prefix,query_prediction,title,tag,label
0,小品,"{""小品大全宋小宝"": ""0.009"", ""小品相亲"": ""0.012"", ""小品剧本"": ...",小品,阅读,0
1,1368,"{""13688cc赛马会"": ""0.059"", ""13685367892"": ""0.124""...",HCG大于1368%2C正常吗,健康,0
2,1368,"{""13688cc赛马会"": ""0.059"", ""13685367892"": ""0.124""...",1368年,百科,1
3,银耳,"{""银耳汤的功效"": ""0.012"", ""银耳为什么不能天天吃"": ""0.009"", ""银耳...",银耳红枣汤的做法,菜谱,1
4,月经量少,"{""月经量少喝红糖水好吗"": ""0.010"", ""月经量少该怎么调理"": ""0.016"", ...",月经量少怎么调理,百科,0


In [39]:
train_df.isnull().sum()

prefix              4
query_prediction    0
title               0
tag                 0
label               0
dtype: int64

In [40]:
train_df['prefix'] = train_df['prefix'].fillna('null')

In [41]:
print('data is null?\n',train_df.isnull().sum())

data is null?
 prefix              0
query_prediction    0
title               0
tag                 0
label               0
dtype: int64


In [42]:
train_df.loc[train_df.prefix == '花开花又', 'query_prediction'] = '{"花开花又落": "0.635", "花开花又落是什么歌": "0.365"}'
train_df.loc[train_df.prefix == '花开花又', 'title'] = '大雁听过我的歌%2C山丹丹花开花又落....一年又一年?谁听过这首歌%2C知道歌名吗?'
train_df.loc[train_df.prefix == '花开花又', 'tag'] = '知道'
train_df.loc[train_df.prefix == '花开花又', 'label'] = 0
# print(train_df[train_df.prefix=='花开花又'])
# add the missed line to end
train_df.loc[1999999, 'prefix'] = '花开花又'
train_df.loc[1999999, 'query_prediction'] = '{"花开花又落": "0.635", "花开花又落是什么歌": "0.365"}'
train_df.loc[1999999, 'title'] = '等你花开花又落'
train_df.loc[1999999, 'tag'] = '音乐'
train_df.loc[1999999, 'label'] = 0
print(train_df[train_df.prefix=='花开花又'])

        prefix                          query_prediction  \
1815101   花开花又  {"花开花又落": "0.635", "花开花又落是什么歌": "0.365"}   
1999999   花开花又  {"花开花又落": "0.635", "花开花又落是什么歌": "0.365"}   

                                               title tag label  
1815101  大雁听过我的歌%2C山丹丹花开花又落....一年又一年?谁听过这首歌%2C知道歌名吗?  知道     0  
1999999                                      等你花开花又落  音乐     0  


In [43]:
train_df.loc[train_df.label == '0', 'label'] = 0
train_df.loc[train_df.label == '1', 'label'] = 1
# print(train_df.label.value_counts())
train_df['label'] = train_df.label.astype('int')

In [44]:
train_df.label.value_counts()

0    1255805
1     744195
Name: label, dtype: int64

In [45]:
pickle.dump(train_df, open(train_preprocessed_file, 'wb'))

In [46]:
# 加载验证集
valid_df = pd.read_csv(valid_file, sep='\t', header=None,
                               names=['prefix', 'query_prediction', 'title', 'tag', 'label'])

In [47]:
valid_df.head()

,prefix,query_prediction,title,tag,label
0,重庆旅游,"{""重庆旅游景区"": ""0.018"", ""重庆旅游攻略"": ""0.373"", ""重庆旅游景点...",皇包车旅行,应用,0
1,婆婆来了,"{""婆婆来了大结局"": ""0.021"", ""婆婆来了电视剧"": ""0.100"", ""婆婆来了...",搜狐视频,应用,0
2,张国荣,"{""张国荣遗体很恐怖"": ""0.020"", ""张国荣头像"": ""0.013"", ""张国荣24...",张国荣,歌手,0
3,陌陌,"{""陌陌晒奶"": ""0.008"", ""陌陌下载安装"": ""0.009"", ""陌陌交友"": ""...",陌陌,百科,0
4,畅游,"{""畅游阁"": ""0.322"", ""畅游六零年代"": ""0.134"", ""畅游书城82212...",李畅游,百科,0


In [48]:
valid_df['label'] = valid_df.label.astype('int')

In [49]:
pickle.dump(valid_df, open(valid_preprocessed_file, 'wb'))

In [50]:
# 加载测试集
test_df = pd.read_csv(test_A_file, sep='\t', header=None, names=['prefix', 'query_prediction', 'title', 'tag'])

In [51]:
test_df.sample(10)

,prefix,query_prediction,title,tag
36906,中南财,"{""中南财经大学"": ""0.045"", ""中南财经政法大学"": ""0.769"", ""中南财经...",中南财经政法大学,百科
33390,作,"{""作业帮免费下载"": ""0.025"", ""作业帮"": ""0.141"", ""作文"": ""0....",作业帮,应用
38612,uu,"{""uu手游加速器下载"": ""0.009"", ""uu加加速器"": ""0.022"", ""uu加...",UU跑腿,应用
31765,都市逆袭,"{""都市逆袭之王txt下载"": ""0.152"", ""都市逆袭之王"": ""0.500"", ""都...",都市逆袭女王,阅读
3928,taob,"{""taobaocom淘宝电脑版"": ""0.019"", ""taobao .com"": ""0....",taobao上的玛卡好不，我早泄了,健康
9282,qq浏览,"{""qq浏览噐下载"": ""0.012"", ""qq浏览器官方下载"": ""0.006"", ""qq...",2345浏览器,应用
1069,考斯特,"{""考斯特房车价格"": ""0.075"", ""考斯特商务车"": ""0.066"", ""考斯特中巴...",丰田考斯特,百科
26215,号码,"{""号码差值"": ""0.089"", ""号码归属地"": ""0.030"", ""号码百事通"": ""...",周易手机号码吉凶测试靠谱吗？看了某网站手机号码吉凶测试结果都一样。,知道
37906,英语自我介绍,"{""英语自我介绍范文"": ""0.019"", ""英语自我介绍作文"": ""0.111"", ""英语...",英语自我介绍脱口说,百科
37572,双色,"{""双色球"": ""0.155"", ""双色球开奖结"": ""0.053"", ""双色球开奖结果"":...",双色球开奖结果查询,经验


In [52]:
test_df[test_df["prefix"]=='苹果']

,prefix,query_prediction,title,tag
40040,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果官网,网站
40199,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果官网,网站
40301,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果手机怎么关闭系统更新提示,经验
40309,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果,影视
40338,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果手机怎么关闭系统更新提示,经验
...,...,...,...,...
49235,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果手机怎么关闭系统更新提示,经验
49295,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果官网,网站
49568,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果手机怎么关闭系统更新提示,经验
49619,苹果,"{""苹果手机"": ""0.017"", ""苹果8plus"": ""0.022"", ""苹果9什么时候...",苹果官网,网站


In [53]:
test_df['label'] = -1

In [54]:
pickle.dump(test_df, open(test_A_preprocessed_file, 'wb'))

In [55]:
# 2. feature engineering

# items = ['prefix', 'prefix_len', 'pred_len', 'title','title_len', 'tag']
train_df, valid_df, test_df = feature_engineering(train_df, valid_df, test_df, items = ['prefix', 'title', 'tag'])



In [56]:
# average click number for items in feature
a = train_df['label'].sum() / train_df['prefix'].nunique()
# average search number for items in feature
b = train_df['label'].count() / train_df['prefix'].nunique()

In [57]:
train_df['label'].sum()

744195

In [58]:
train_df['prefix'].nunique()

157083

In [59]:
a

4.73759095510017

In [60]:
b

12.732122508482776

In [61]:
# click number & search number for specific value of feature
temp1 = train_df[['prefix', 'label']].groupby('prefix', as_index=False)['label'].agg(
            {'prefix' + '_click': 'sum', 'prefix' + '_count': 'count'})
temp1

,prefix,prefix_click,prefix_count
0,#,13,41
1,%,13,33
2,&,6,14
3,(女人,1,1
4,*p++,0,1
...,...,...,...
157078,龟鹿二仙膏,1,3
157079,龟龙,0,2
157080,사랑해요,1,1
157081,안녕하세요,1,2


In [62]:
# absolutely transform ratio
temp1['prefix' + '_ctr_abs'] = temp1['prefix' + '_click'] / (temp1['prefix' + '_count'])

# smoothly transform ratio  平滑变换比
temp1['prefix' + '_ctr_smooth'] = (temp1['prefix' + '_click'] + a) / (temp1['prefix' + '_count'] + b)

In [63]:
temp1

,prefix,prefix_click,prefix_count,prefix_ctr_abs,prefix_ctr_smooth
0,#,13,41,0.317073,0.330111
1,%,13,33,0.393939,0.387858
2,&,6,14,0.428571,0.401674
3,(女人,1,1,1.000000,0.417823
4,*p++,0,1,0.000000,0.345001
...,...,...,...,...,...
157078,龟鹿二仙膏,1,3,0.333333,0.364705
157079,龟龙,0,2,0.000000,0.321582
157080,사랑해요,1,1,1.000000,0.417823
157081,안녕하세요,1,2,0.500000,0.389461


In [64]:
def create_single_feature(feature, train_df, valid_df, test_data):
    feature_path = './features/' + feature + '.pkl'
    if os.path.exists(feature_path):
        temp = pickle.load(open(feature_path, 'rb'))
    else:
        # average click number for items in feature
        a = train_df['label'].sum() / train_df[feature].nunique()
        # loga = np.log1p(train_df['label'].sum())/np.log(train_df[feature].nunique())

        # average search number for items in feature
        b = train_df['label'].count() / train_df[feature].nunique()
        # logb = np.log1p(train_df['label'].count())/np.log1p(train_df[feature].nunique())

        # click number & search number for specific value of feature
        temp = train_df[[feature, 'label']].groupby(feature, as_index=False)['label'].agg(
            {feature + '_click': 'sum', feature + '_count': 'count'})
        # temp[feature+'_click_smooth'] = (temp[feature+'_click']+a)/2
        # temp[feature+'_count_smooth'] = (temp[feature+'_count']+b)/2

        # log transform of click & search number
        # temp[feature+'_click_log'] = np.log1p(temp[feature+'_click'])
        # temp[feature+'_count_log'] = np.log1p(temp[feature+'_count'])

        # twice log transform of click & search number
        # temp[feature+'_click_log2'] = np.log1p(temp[feature+'_click_log'])
        # temp[feature+'_count_log2'] = np.log1p(temp[feature+'_count_log'])

        # diff for click & search number
        # temp[feature+'_click_diff'] = temp[feature+'_click'] - temp[feature+'_click'].mean()
        # temp[feature+'_count_diff'] = temp[feature+'_count'] - temp[feature+'_count'].mean()

        # ratio for click & search number
        # temp[feature+'_click_ratio'] = temp[feature+'_click']/temp[feature+'_click'].sum()
        # temp[feature+'_count_ratio'] = temp[feature+'_count']/temp[feature+'_count'].sum()

        # ratio for log of click & search number
        # temp[feature+'_click_ratio_log'] = np.log1p(temp[feature+'_click'])-np.log1p(temp[feature+'_click'].sum())
        # temp[feature+'_count_ratio_log'] = np.log1p(temp[feature+'_count'])-np.log1p(temp[feature+'_count'].sum())

        # absolutely transform ratio
        temp[feature + '_ctr_abs'] = temp[feature + '_click'] / (temp[feature + '_count'])

        # smoothly transform ratio
        temp[feature + '_ctr_smooth'] = (temp[feature + '_click'] + a) / (temp[feature + '_count'] + b)

        # diff of absolutely transform ratio
        # temp[feature+'_ctr_diff'] = temp[feature+'_ctr_abs'] - temp[feature+'_ctr_abs'].mean()

        # relatively transform ratio
        # temp[feature+'_ctr_rela'] = temp[feature+'_ctr_diff']/(temp[feature+'_ctr_abs'].mean())

        # log of absolutely transform ratio
        # temp[feature+'_ctr_log_abs'] = temp[feature+'_click_log']-(temp[feature+'_count_log'])

        # log of smoothly transform ratio
        # temp[feature+'_ctr_log_smooth'] = (temp[feature+'_click_log']+np.log1p(a))/(temp[feature+'_count_log']+np.log1p(b))

        # log of diff of absolutely transform ratio
        # temp[feature+'_ctr_log_diff'] = temp[feature+'_ctr_log_abs'] - temp[feature+'_ctr_log_abs'].mean()

        # log of relatively transform ratio
        # temp[feature+'_ctr_log_rela'] = temp[feature+'_ctr_log_diff']/(temp[feature+'_ctr_log_abs'].mean())

        pickle.dump(temp, open(feature_path, 'wb'))

    return temp
def create_feature_pair(item_g, train_df, valid_df, test_data):
    feature_path = './features/' + '_'.join(item_g) + '.pkl'
    if os.path.exists(feature_path):
        temp = pickle.load(open(feature_path, 'rb'))
    else:
        a = train_df['label'].sum() / train_df[item_g].nunique()
        b = train_df['label'].count() / train_df[item_g].nunique()
        temp = train_df.groupby(item_g, as_index=False)['label'].agg(
            {'_'.join(item_g) + '_click': 'sum', '_'.join(item_g) + '_count': 'count'})
        # # 平滑点击数与搜索数
        # temp['_'.join(item_g)+'_click_smooth'] = temp['_'.join(item_g)+'_click']+a
        # temp['_'.join(item_g)+'_count_smooth'] = temp['_'.join(item_g)+'_count']+b
        # # 点击数和搜索数的log变换
        # temp['_'.join(item_g)+'_click_log'] = np.log1p(temp['_'.join(item_g)+'_click'])
        # temp['_'.join(item_g)+'_count_log'] = np.log1p(temp['_'.join(item_g)+'_count'])

        # # 点击数和搜索数与均值的差值
        # temp['_'.join(item_g)+'_click_diff'] = temp['_'.join(item_g)+'_click'] - temp['_'.join(item_g)+'_click'].mean()
        # temp['_'.join(item_g)+'_count_diff'] = temp['_'.join(item_g)+'_count'] - temp['_'.join(item_g)+'_count'].mean()

        # # 点击数占比与搜索数占比
        # temp['_'.join(item_g)+'_click_ratio'] = temp['_'.join(item_g)+'_click']/temp['_'.join(item_g)+'_click'].sum()
        # temp['_'.join(item_g)+'_count_ratio'] = temp['_'.join(item_g)+'_count']/temp['_'.join(item_g)+'_count'].sum()
        # # 平滑点击数占比与搜索数占比
        # temp['_'.join(item_g)+'_click_ratio_smooth'] = (temp['_'.join(item_g)+'_click']+a)/(2*temp['_'.join(item_g)+'_click'].sum())
        # temp['_'.join(item_g)+'_count_ratio_smooth'] = (temp['_'.join(item_g)+'_count']+b)/(2*temp['_'.join(item_g)+'_count'].sum())
        # # 点击数log占比与搜索数log占比
        # temp['_'.join(item_g)+'_click_ratio_log'] = np.log1p(temp['_'.join(item_g)+'_click_ratio'])
        # temp['_'.join(item_g)+'_count_ratio_log'] = np.log1p(temp['_'.join(item_g)+'_count_ratio'])

        # 绝对转化率
        temp['_'.join(item_g) + '_ctr_abs'] = temp['_'.join(item_g) + '_click'] / (temp['_'.join(item_g) + '_count'])
        # 贝叶斯平滑转换率
        temp['_'.join(item_g) + '_ctr_smooth'] = (temp['_'.join(item_g) + '_click'] + a) / (
                    temp['_'.join(item_g) + '_count'] + b)
        # # 绝对转化率差值
        # temp['_'.join(item_g)+'_ctr_diff'] = temp['_'.join(item_g)+'_ctr_abs'] - temp['_'.join(item_g)+'_ctr_abs'].mean()
        # # 相对转化率
        # temp['_'.join(item_g)+'_ctr_rela'] = temp['_'.join(item_g)+'_ctr_diff']/(temp['_'.join(item_g)+'_ctr_abs'].mean())

        # # prefix的log绝对转换率
        # temp['_'.join(item_g)+'_ctr_log_abs'] = temp['_'.join(item_g)+'_click_log']-(temp['_'.join(item_g)+'_count_log'])
        # # prefix的log贝叶斯平滑转换率
        # temp['_'.join(item_g)+'_ctr_log_smooth'] = (temp['_'.join(item_g)+'_click_log']+np.log1p(a))/(temp['_'.join(item_g)+'_count_log']+np.log1p(b))
        # # prefix的log绝对转化率差值
        # temp['_'.join(item_g)+'_ctr_log_diff'] = temp['_'.join(item_g)+'_ctr_log_abs'] - temp['_'.join(item_g)+'_ctr_log_abs'].mean()
        # # prefix的log相对转化率
        # temp['_'.join(item_g)+'_ctr_log_rela'] = temp['_'.join(item_g)+'_ctr_log_diff']/(temp['_'.join(item_g)+'_ctr_log_abs'].mean())

        pickle.dump(temp, open(feature_path, 'wb'))
    return temp


In [67]:
train_df, valid_df, test_df = feature_engineering(train_df, valid_df, test_df, items = ['prefix', 'title', 'tag'])

,prefix,query_prediction,title,tag,label,prefix_click_x,prefix_count_x,prefix_ctr_abs_x,prefix_ctr_smooth_x,title_click_x,...,prefix_title_ctr_abs_y,prefix_title_ctr_smooth_y,prefix_tag_click_y,prefix_tag_count_y,prefix_tag_ctr_abs_y,prefix_tag_ctr_smooth_y,title_tag_click_y,title_tag_count_y,title_tag_ctr_abs_y,title_tag_ctr_smooth_y
0,小品,"{""小品大全宋小宝"": ""0.009"", ""小品相亲"": ""0.012"", ""小品剧本"": ...",小品,阅读,0,58,150,0.386667,0.385527,36,...,0.382979,NaN,3,46,0.065217,NaN,3,46,0.065217,NaN
1,1368,"{""13688cc赛马会"": ""0.059"", ""13685367892"": ""0.124""...",HCG大于1368%2C正常吗,健康,0,1,2,0.500000,0.389461,0,...,0.000000,NaN,0,1,0.000000,NaN,0,1,0.000000,NaN
2,1368,"{""13688cc赛马会"": ""0.059"", ""13685367892"": ""0.124""...",1368年,百科,1,1,2,0.500000,0.389461,1,...,1.000000,NaN,1,1,1.000000,NaN,1,1,1.000000,NaN
3,银耳,"{""银耳汤的功效"": ""0.012"", ""银耳为什么不能天天吃"": ""0.009"", ""银耳...",银耳红枣汤的做法,菜谱,1,205,568,0.360915,0.361161,126,...,0.634021,NaN,123,194,0.634021,NaN,126,203,0.620690,NaN
4,月经量少,"{""月经量少喝红糖水好吗"": ""0.010"", ""月经量少该怎么调理"": ""0.016"", ...",月经量少怎么调理,百科,0,68,185,0.367568,0.367859,37,...,0.469697,NaN,31,66,0.469697,NaN,37,73,0.506849,NaN


In [68]:
# 3.1 Construct training set, validation set and test set
def split_data(train_df, valid_df, test_df):
    # train_df = train_df.drop(['prefix','query_prediction', 'pred', 'title', 'tag'], axis = 1)
    # valid_df = valid_df.drop(['prefix','query_prediction', 'pred', 'title', 'tag'], axis = 1)
    # test_df = test_df.drop(['prefix','query_prediction', 'pred', 'title', 'tag'], axis = 1)
    train_df = train_df.drop(['prefix', 'query_prediction', 'title', 'tag'], axis=1)
    valid_df = valid_df.drop(['prefix', 'query_prediction', 'title', 'tag'], axis=1)
    test_df = test_df.drop(['prefix', 'query_prediction', 'title', 'tag'], axis=1)

    X = train_df.drop(['label'], axis=1)
    y = train_df['label']
    X_valid = valid_df.drop(['label'], axis=1)
    y_valid = valid_df['label']
    X_test = test_df.drop(['label'], axis=1)

    del train_df, valid_df, test_df

    return X, y, X_valid, y_valid, X_test

In [69]:
# 3. training model

X, y, X_valid, y_valid, X_test = split_data(train_df, valid_df, test_df)

In [70]:
# 3.2 train model
def train_lgb_model(X, y, X_valid, y_valid, params, model_path):
    lgb_train = lgb.Dataset(X, y)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=5000,
                    valid_sets=[lgb_eval, lgb_train],
                    valid_names=['eval', 'train'],
                    early_stopping_rounds=50,
                    verbose_eval=True,
                    )

    # f1-score on train data
    print('f1-score on train data:',
          f1_score(y, np.where(gbm.predict(X, num_iteration=gbm.best_iteration) > 0.5, 1, 0)))
    # f1-score on validation data
    print('f1-score on validation data:',
          f1_score(y_valid, np.where(gbm.predict(X_valid, num_iteration=gbm.best_iteration) > 0.5, 1, 0)))
    print(gbm.best_score)

    joblib.dump(gbm, model_path)
    # gbm.save_model(model_file, num_iteration=gbm.best_iteration)
    return gbm


def load_model(X, y, X_valid, y_valid, params, model_path):
    if os.path.exists(model_path):
        gbm = joblib.load(model_path)
    else:
        gbm = train_lgb_model(X, y, X_valid, y_valid, params, model_path)

    return gbm

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 32,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1
}
model_path = './models/lgbm.pkl'
gbm = load_model(X, y, X_valid, y_valid, params, model_path)

TypeError: Wrong type(Series) for label.
It should be list, numpy 1-D array or pandas Series